### Trying out RAG with ollama and chromadb
ollama is installed in the python environment venvcrc5 from where this notebook is started.

ollama is recommended over hugging face for local experimentation

it uses a docker-like syntax

In [5]:
!ollama run deepseek-v3.1:671b-cloud

⠙ You need to be signed in to Ollama to run Cloud models.

To sign in, navigate to:
    https://ollama.com/connect?name=nuc11&key=c3NoLWVkMjU1MTkgQUFBQUMzTnphQzFsWkRJMU5URTVBQUFBSUtRSUZNMHZoS1RtNEpmS2hiWXVodjVuMmcyKytIS0NxcktHL3VlSmNpY3k



In [1]:
!ollama ls

NAME                        ID              SIZE      MODIFIED     
deepseek-r1:14b             c333b7232bdb    9.0 GB    19 hours ago    
nomic-embed-text:latest     0a109f422b47    274 MB    19 hours ago    
deepseek-v3.1:671b-cloud    d3749919e45f    -         25 hours ago    
deepseek-r1:1.5b            e0979632db5a    1.1 GB    8 weeks ago     
deepseek-r1:7b              755ced02ce7b    4.7 GB    8 weeks ago     
gemma3:12b                  6fd036cefda5    8.1 GB    6 months ago    
deepseek-r1:latest          0a8c26691023    4.7 GB    7 months ago    
llama3.1:latest             46e0c10c039e    4.9 GB    7 months ago    


In [2]:
!ollama
!ollama --version
!ollama list

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  signin      Sign in to ollama.com
  signout     Sign out from ollama.com
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.
ollama version is 0.12.0
NAME                        ID              SIZE      MODIFIED     
deepseek-r1:14b             c333b7232bdb    9.0 GB    19 hours ago    
nomic-embed-text:latest     0a109f422b47    274 MB    19 hours ago    
deepseek-v3.1:671b-cloud    d3749919e45f    -         25 hours ago    
deep

In [2]:
!ollama help pull

Pull a model from a registry

Usage:
  ollama pull MODEL [flags]

Flags:
  -h, --help       help for pull
      --insecure   Use an insecure registry

Environment Variables:
      OLLAMA_HOST                IP Address for the ollama server (default 127.0.0.1:11434)


#### Pull the cloud-hosted deepseek-v3.1:671b-cloud model (experimental, not used)

In [4]:
!ollama pull deepseek-v3.1:671b-cloud

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         pulling manifest 
pulling 8aacb627728e: 100% ▕██████████████████▏  405 B                         
verifying sha2

In [2]:
!ollama ls

NAME                        ID              SIZE      MODIFIED       
deepseek-r1:14b             c333b7232bdb    9.0 GB    12 minutes ago    
nomic-embed-text:latest     0a109f422b47    274 MB    12 minutes ago    
deepseek-v3.1:671b-cloud    d3749919e45f    -         6 days ago        
deepseek-r1:1.5b            e0979632db5a    1.1 GB    2 months ago      
deepseek-r1:7b              755ced02ce7b    4.7 GB    2 months ago      
gemma3:12b                  6fd036cefda5    8.1 GB    6 months ago      
deepseek-r1:latest          0a8c26691023    4.7 GB    7 months ago      
llama3.1:latest             46e0c10c039e    4.9 GB    7 months ago      


#### The pdfreader translates any pdf document to text readable by the model

In [2]:
from pypdf import PdfReader
# my textbook, 5th ed
reader = PdfReader("/home/mort/LaTeX/new projects/CRC5/main.pdf")
total_pages = len(reader.pages)
all_text = ""
for page_num in range(total_pages):
    page = reader.pages[page_num]
    all_text += page.extract_text()
f = open("/home/mort/crc5pdfreader/main.txt", "w")
f.write(all_text)
f.close()

#### Here the original LaTeX files are collected instead

In [1]:
import glob
# Find `.tex` files in LaTeX directory and all subdirectories
tex_files = glob.glob('/home/mort/LaTeX/new projects/CRC5/**/chapter[1-9].tex', recursive = True)
tex_files.sort()
f = open("/home/mort/crc5latex/main.txt", "w")
for file in tex_files:
    g = open(file, "r")
    content = g.read()
    f.write(content)
    g.close()
f.close()    
    

#### Aside :Using crawl4ai to scrape web pages (writes content to /home/mort/crawl4ai.md)

In [ ]:
!python3 scripts/testcrawler.py "http://www.imm.dtu.dk/~alan/publications.html" 1

#### Note: The LlamaParse version of the text pdf (parsed from the web API: https://www.llamaindex.ai/llamaparse) was stored in /home/mort/crc5llamaparse. This version is used in what follows.

#### Code for preprocessing the RAG supplementary text

In [9]:
import os
import re
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter

def readtextfiles(path):
    text_contents = {}
    directory = os.path.join(path)
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
        
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
        
            text_contents[filename] = content
        
        return text_contents

def chunksplitter(text, chunk_size=512, chunk_overlap=128):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # Desired chunk size in tokens
        chunk_overlap=chunk_overlap,  # Overlap between chunks
        separators=["\n\n", "\n", " "]  # Split by paragraphs, then sentences, then words
    )
    return splitter.split_text(text)

# use the nomic-embed-text model to calculate vector embeddings for all text chunks
def getembedding(chunks):
    embeds = ollama.embed(model="nomic-embed-text", input=chunks)
    return embeds.get('embeddings', [])

#### Add the supplementary text to a new database collection
The embedded chunk vectors are always 768 bytes. Don't rerun unles using different chunk sizes

In [10]:
import chromadb
import numpy as np
#chromaclient = chromadb.PersistentClient(path="/home/mort/crc5imagery/crc5rag")
chromaclient = chromadb.PersistentClient(path="/home/mort/crc5rag")
chromaclient.delete_collection("crc5rag")
collection = chromaclient.create_collection( name="crc5rag", metadata={"hnsw:space": "cosine"} )

# the RAG supplementary data, here using the llamaparse version of the main pdf
textdocspath = "/home/mort/crc5llamaparse"
text_data = readtextfiles(textdocspath)

# read, break into chunks, embed and add to the chroma vector database 
for filename, text in text_data.items():
    # default chunk size 512, overlap 128 
    chunks = chunksplitter(text)
    embeds = getembedding(chunks)
    chunknumber = list(range(len(chunks)))
    ids = [filename + str(index) for index in chunknumber]
    metadatas = [{"source": filename} for index in chunknumber]
    collection.add(ids=ids, documents=chunks, embeddings=embeds, metadatas=metadatas)

#### Execute a query with a deepseek model and the supplementary text (RAG)

In [1]:
%%capture
!ollama pull nomic-embed-text
#!ollama pull deepseek-v3.1:671b-cloud
!ollama pull deepseek-r1:14b

In [3]:
!ollama list

NAME                        ID              SIZE      MODIFIED     
nomic-embed-text:latest     0a109f422b47    274 MB    4 days ago      
deepseek-r1:14b             c333b7232bdb    9.0 GB    4 days ago      
deepseek-v3.1:671b-cloud    d3749919e45f    -         11 days ago     
deepseek-r1:1.5b            e0979632db5a    1.1 GB    2 months ago    
deepseek-r1:7b              755ced02ce7b    4.7 GB    2 months ago    
gemma3:12b                  6fd036cefda5    8.1 GB    6 months ago    
deepseek-r1:latest          0a8c26691023    4.7 GB    7 months ago    
llama3.1:latest             46e0c10c039e    4.9 GB    8 months ago    


In [6]:
import gradio as gr
import html
import ollama
import chromadb

# Initialize ChromaDB client and collection
#chromaclient = chromadb.PersistentClient(path="/home/mort/crc5imagery/crc5rag")
chromaclient = chromadb.PersistentClient(path="/home/mort/crc5rag")
collection = chromaclient.get_collection(name="crc5rag")

def ragask(query):
    # Embed the query
    queryembed = ollama.embed(model="nomic-embed-text", input=query)['embeddings']
    # Retrieve related documents (16 chunks)
    relateddocs = '\n\n'.join(collection.query(query_embeddings=queryembed, n_results=16)['documents'][0])

    # view chunked related docs
    print(len(relateddocs))
    print(relateddocs)
    
    # Generate a response
    prompt = f"Answer the question: {query} referring to the following text as a resource: {relateddocs}"
    # using deepseek-r1:14b
    response = ollama.generate(model="deepseek-r1:14b", prompt=prompt, stream=False, think=False)['response']   
    # Ensure the response is valid Markdown
    return html.escape(response)

# Launch Gradio Interface (ChatInterface not appropriate for RAG application!)
gr.Interface(fn=ragask,inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
             outputs="markdown",
             description="Ask questions about the book contents",
             title="Image Analysis, Classification and Change Detection in Remote Sensing").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


7503
mixture model (the feed-forward neural network or FFN). In the case of the
neural network, the back propagation      training algorithm is by far the most
commonly used. We make some considerable eﬀort to explain it in detail and,
in Appendix B, to develop more sophisticated and eﬃcient analytical methods
based on scaled conjugate gradient and the Kalman ﬁlter. All of these, how-
ever, will be seen to become unwieldy for so-called deep learning or multiple

---
B
Neural Network Training Algorithms
The gradient descent back-propagation algorithm introduced in Chapter 6 is
notoriously slow to converge. In this appendix, we will develop two additional
training algorithms for the two-layer, feed-forward neural network of Figure
6.11. The ﬁrst of these, the scaled conjugate gradient, makes use of the second
derivatives of the cost function with respect to the synaptic weights, i.e., of
the Hessian matrix. The second, the extended Kalman ﬁlter method, takes

Neural networks             